In [3]:
import dependencias

pygame 2.6.1 (SDL 2.28.4, Python 3.12.1)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [14]:
agent = dependencias.Agentes.ChaserAgent()
game = dependencias.Snake_game((15, 15), 5, agent)
game.evaluar()

100%|██████████| 100/100 [00:00<00:00, 1305.73it/s]


{'puntuacion_media': np.float64(8.07),
 'puntuacion_maxima': np.int64(25),
 'puntuacion_minima': np.int64(3),
 'movimientos_medios': np.float64(37.36),
 'movimientos_maximos': np.int64(122),
 'movimientos_minimos': np.int64(9),
 'movimientos por puntuacion': np.float64(4.629491945477075)}

In [7]:
agent = dependencias.Agentes.Avoid_inmediate_death()
game = dependencias.Snake_game((15, 15), 5, agent)
game.play_with_pygame()


In [12]:
agent = dependencias.Agentes.Avoid_inmediate_death()
game = dependencias.Snake_game((15, 15), 5, agent)
game.evaluar()


100%|██████████| 100/100 [00:00<00:00, 486.24it/s]


{'puntuacion_media': np.float64(8.45),
 'puntuacion_maxima': np.int64(16),
 'puntuacion_minima': np.int64(4),
 'movimientos_medios': np.float64(510.44),
 'movimientos_maximos': np.int64(1154),
 'movimientos_minimos': np.int64(163),
 'movimientos por puntuacion': np.float64(60.40710059171598)}

In [13]:
agent_1 = dependencias.Agentes.ChaserAgent()
agente_2 = dependencias.Agentes.Avoid_inmediate_death()

agente_combiando = dependencias.Agentes.Combined_agent(agentes = (agent_1, agente_2), weights= (0.5, 1))

game = dependencias.Snake_game((15, 15), 5, agente_combiando)
game.evaluar()

100%|██████████| 100/100 [00:00<00:00, 211.89it/s]


{'puntuacion_media': np.float64(32.38),
 'puntuacion_maxima': np.int64(63),
 'puntuacion_minima': np.int64(8),
 'movimientos_medios': np.float64(177.65),
 'movimientos_maximos': np.int64(393),
 'movimientos_minimos': np.int64(37),
 'movimientos por puntuacion': np.float64(5.486411365040148)}

In [6]:
agent = dependencias.Agentes.Aantiloop()
game = dependencias.Snake_game((15, 15), 30, agent)
game.play_with_pygame()


In [ ]:
combinado = dependencias.Agentes.Combined_agent(agentes = (agente_avoider, agen), weights= (0.5, 1))